In [1]:
import json
import os
from PIL import Image, ImageDraw, ImageFont
from moviepy.editor import *
from moviepy.video.tools.subtitles import SubtitlesClip
from src.ai import elevenlabs, lemonfox
from src.utils import saveB64Audio, create_project_folder
import sys

sys.path.append('/Users/user/Code/GitHub/comic-generator-ai/moviepy')

def create_subtitle_clip(txt, font, fontsize, color):
    clip = TextClip(txt, fontsize=fontsize, font=font, color=color)
    clip = clip.on_color(size=(clip.w + 10, clip.h + 10), color=(0,0,0,0), pos=(5, 5))
    return clip.set_pos(('center', 'bottom'))

def create_video(script, width=1280, height=720, fps=30, font_size=40, font_color='white'):
    project_folder = create_project_folder()

    # Save audio
    audio = elevenlabs.getSpeechB64(script)
    audio_path = os.path.join(project_folder, 'audio.mp3')
    saveB64Audio(audio, audio_path)

    # Get transcript
    transcript = lemonfox.getTranscript(audio_path)
    transcript_json = transcript.model_dump_json()
    transcript_path = os.path.join(project_folder, 'transcript.json')
    with open(transcript_path, 'w') as f:
        f.write(transcript_json)

    # Load the transcript JSON
    with open(transcript_path, 'r') as f:
        transcript = json.load(f)

    # Create subtitle clips
    subtitle_clips = []
    for segment in transcript['segments']:
        start, end = segment['start'], segment['end']
        text = segment['text']
        subtitle_clip = create_subtitle_clip(text, 'Arial', font_size, font_color)
        subtitle_clip = subtitle_clip.set_start(start).set_end(end)
        subtitle_clips.append(subtitle_clip)

    # Create a CompositeVideoClip with subtitles
    video = CompositeVideoClip([ColorClip(size=(width, height), color=(0,0,0,0)).set_duration(transcript['segments'][-1]['end'])] + subtitle_clips)

    # Load audio
    audio = AudioFileClip(audio_path)

    # Set video audio
    video = video.set_audio(audio)

    # Write the result to a file
    output_path = os.path.join(project_folder, 'output.webm')
    video.write_videofile(output_path, fps=fps, codec='libvpx-vp9', audio_codec='libvorbis')

    print(f"Video created successfully in {project_folder}")
    return project_folder, output_path

# Example usage
script = """
This should sound better -- using the nicer voice model here. It costs more, so I use the other model for the speed and lower price.

We're using ElevenLabs' cheaper model for the TTS, and LemonFox's Whisper API to get the transcript.

Kdenlive was used to bring the two together to create a video. Now there just needs to be some visuals for a full video.
"""

create_video(
    script,
    width=1280,
    height=720,
    fps=30,
    font_size=60,  # Increased font size
    font_color='white'  # White color
)

Moviepy - Building video project_20241014_014526/output.webm.
MoviePy - Writing audio in outputTEMP_MPY_wvf_snd.ogg


MoviePy - Done.
Moviepy - Writing video project_20241014_014526/output.webm



Moviepy - Done !
Moviepy - video ready project_20241014_014526/output.webm
Video created successfully in project_20241014_014526


('project_20241014_014526', 'project_20241014_014526/output.webm')